#### 3.2. Capturing game state and checking for illegal moves


Now that you’ve implemented the rules for placing and capturing stones on a `Board`, let’s move on to playing games by capturing the current state of a game in a `GameState` class. Roughly speaking, __game state__ knows about the board position, the next player, the previous game state, and the last move that has been played. What follows is just the beginning of the definition. You’ll add more functionality to `GameState` throughout this section. Again, you put this into goboard_slow.py.



In [27]:
# Listing 3.10. Encoding game state for a game of Go
class GameState():
    """ 
    >>> from dlgo.goboard_fast import Move,GoString,Board,GameState
    >>> g=get_game_state()
    >>> gg=g.new_game(19)
    >>> isinstance(gg,GameState)
    True
    """
    def __init__(self, board, next_player, previous, move):
        self.board = board
        self.next_player = next_player
        self.previous_state = previous
        self.last_move = move

    def apply_move(self, move):
        if move.is_play:
            next_board = copy.deepcopy(self.board)
            next_board.place_stone(self.next_player, move.point)
        else:
            next_board = self.board
        return GameState(next_board, self.next_player.other, self, move)

    @classmethod
    def new_game(cls, board_size=19):
        if isinstance(board_size, int):
            board_size = (board_size, board_size)
        board = Board(*board_size)
        return GameState(board, Player.black, None, None)



At this point, you can already decide when a game is over by adding the following code to your `GameState` class.



In [28]:
# Listing 3.11. Deciding when a game of Go is over
def is_over(self):
    if self.last_move is None:
        return False
    if self.last_move.is_resign:
        return True
    second_last_move = self.previous_state.last_move
    if second_last_move is None:
        return False
    return self.last_move.is_pass and second_last_move.is_pass

Now that you’ve implemented how to apply a move to the current game state, using apply_move, you should also write code to identify which moves are legal. Humans may accidentally attempt an illegal move. Bots might attempt illegal moves because they don’t know any better. You need to check three rules:

- Confirm that the point you want to play is empty.
- Check that the move isn’t a self-capture.
- Confirm that the move doesn’t violate the ko rule.

Although the first point is trivial to implement, the other two deserve separate treatment, because they’re rather tricky to handle properly.

##### 3.2.1. Self-capture
When a string of your stones has only one liberty left and you play at the point that removes that liberty, we call that a `self-capture`. For instance, in figure 3.3, the black stones are doomed.

__Figure 3.3. In this Go board state, the three black stones have one liberty left; namely, the marked point. You enforce the self-capture rule, so black isn’t allowed to play there. White, on the other hand, can capture the three black stones by playing on the marked point.__
<p align="center">
     <img src="https://drek4537l1klr.cloudfront.net/pumperla/Figures/03fig03.jpg" alt="string">
</p>


White can capture them at any time by playing on the marked point, and black has no way to prevent it. But what if black played on the marked point? The entire group would have no liberties and would then be captured. Most rule sets forbid such a play, although a few exceptions exist. Most notably, self-capture is allowed in the quadrennial Ing Cup, which is one of the biggest prizes in international Go.

You’ll enforce the self-capture rule in your code. This is consistent with the most popular rule sets, and it also reduces the number of moves your bots need to consider. It’s possible to contrive a position where self-capture is the best move, but such situations are basically unheard of in serious games.

If you alter the surrounding stones in figure 3.3 slightly, you end up with a completely different situation, shown in figure 3.4.

__Figure 3.4. In this situation, the marked point is a capture for black, not suicide, because black will capture two white stones and thereby regain two liberties.__
<p align="center">
     <img src="https://drek4537l1klr.cloudfront.net/pumperla/Figures/03fig04.jpg" alt="string">
</p>

Note that in figure 3.4, and in general, you must remove opponent stones first before checking whether the newly played stone has any liberties. In all rule sets, this next move is a valid capture, not a self-capture, because black will regain two liberties by capturing the two white stones.

Note that the `Board` class does permit self-capture, but in __GameState__ you’ll enforce the rule by applying the move to a copy of the board and checking the number of liberties afterward.

In [29]:
# Listing 3.12. Continuing our definition of GameState to enforce the self-capture rule
def is_move_self_capture(self, player, move):
    if not move.is_play:
        return False
    next_board = copy.deepcopy(self.board)
    next_board.place_stone(player, move.point)
    new_string = next_board.get_go_string(move.point)
    return new_string.num_liberties == 0

##### 3.2.2. Ko
Having checked for self-capture, you can now move on to implement the ko rule. Chapter 2 briefly covered ko and its importance in the game of Go. Roughly speaking, the ko rule applies if a move would return the board to the exact previous position. This doesn’t imply that a player can’t hit back immediately, as the following sequence of diagrams shows. In figure 3.5, white has just played the isolated stone on the bottom. Black’s two stones now have only a single liberty left—but so does this white stone.

In [30]:
import doctest
from summary import get_game_state
doctest.testmod()

TestResults(failed=0, attempted=4)